In [12]:

from dotenv import dotenv_values
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chains import LLMChain
from langchain.schema import HumanMessage, SystemMessage, AIMessage

# Load API key from .env
env_values = dotenv_values("./app.env")
openai_api_key = env_values.get("OPENROUTER_API_KEY")



# Initialize the model , openai/gpt-4o-mini
llm = ChatOpenAI(
    model_name="openai/gpt-oss-20b:free",
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key=openai_api_key,
)


## Simple Example of FewShotPromptTemplate

In [8]:
demo_message = """
Country Name: Egypt | Capital Name: Cairo
Country Name: Saudi Arabia | Capital Name: Riyadh
Country Name: France | Capital Name: Paris
Country Name: China |
""".strip()

# Correct way: Pass a list of message objects
response = llm([HumanMessage(content=demo_message)])
print(response)

content='Country Name: China | Capital Name: Beijing' additional_kwargs={} example=False


## Full Example of FewShotPromptTemplate

In [22]:
from langchain.prompts.few_shot import FewShotPromptTemplate

demo_template = PromptTemplate(
    template="Country Name: {country} | Capital Name: {capital}",
    input_variables=['country', 'capital']
)

examples = [
    {"country": "Egypt", "capital": "Cairo"},
    {"country": "Saudi Arabia", "capital": "Riyadh"},
    {"country": "France", "capital": "Paris"},
]

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=demo_template,
    suffix="Country Name: {country} |",
    input_variables=['country']
)

user_country = 'mexico'  # Note: You had a typo - "Astria" instead of "Austria"

# Format the prompt
formatted_prompt = prompt.format(country=user_country)

# Wrap it in a HumanMessage and pass as a list
response = llm([HumanMessage(content=formatted_prompt)])
print(response.content)

Here’s the complete list with the missing capital filled in:

| Country Name   | Capital Name   |
|----------------|----------------|
| Egypt          | Cairo          |
| Saudi Arabia   | Riyadh         |
| France         | Paris          |
| Mexico         | Mexico City    |

Let me know if you need any additional information or formatting!


## Anther Example

In [27]:
examples = [
    {"recipe": "olives pizza", "ingredients": "\n".join([
                                    "* 1/2 pizza dough recipe",
                                    "* 1 tablespoon olive oil",
                                    "* 8 ounces cream cheese",
                                    "* 1/2 cup mozzarella cheese shredded",
                                    "* 1 tablespoon fresh parsley",
                                    "* 1/3 cup sliced green olives",
                                    "* 1/3 cup sliced black olives"
                                ])},
    {"recipe":"falafel", "ingredients": "\n".join(
                                [
                                    "* 1 cup dried chickpeas, soaked overnight",
                                    "* 1/2 cup onion",
                                    "* 1 cup parsley",
                                    "* 1 cup cilantro",
                                    "* 1 tablespoon fresh parsley",
                                    "* 1/3 cup sliced green olives",
                                    "* 1/3 cup sliced black olives",
                                    "* 3 garlic cloves",
                                    "* 1 tsp salt"
                                ]
                                )}
]

example_prompt = PromptTemplate(
    template="Recipe Name: {recipe}\nIngredients: {ingredients}",
    input_variables=['recipe', 'ingredients']
)

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Recipe Name: {recipe}\n",
    input_variables=['recipe']
)

user_recipe = 'mashed potato'

message = prompt.format(recipe=user_recipe)

response = llm([HumanMessage(content=message)])
print(response.content)

It looks like the mashed‑potato recipe was left unfinished.  Below is a classic, well‑balanced mashed‑potato recipe you can add to the list.  Feel free to tweak any ingredient or amount to suit your taste or dietary needs.

---

## Mashed Potato

### Ingredients  
| Ingredient | Amount | Notes |
|------------|--------|-------|
| **Potatoes** | 2 lb (about 900 g) | Yukon Gold or Russet; peeled and cut into 1‑inch cubes |
| **Water** | 1 cup | For boiling the potatoes |
| **Unsalted butter** | 4 Tbsp (about 60 g) | Room‑temperature |
| **Heavy cream** | ½ cup (120 ml) | Or milk for a lighter version |
| **Salt** | ½ tsp (to taste) | Add more after mashing |
| **Freshly ground black pepper** | ¼ tsp | To taste |
| **Optional extras** | |  |
|  | 1 clove garlic, minced | Adds a subtle garlicky flavor |
|  | 2 Tbsp chopped chives or green onions | For garnish |
|  | 1 Tbsp grated Parmesan | For a cheesy kick |
|  | 1 Tbsp sour cream or Greek yogurt | For extra creaminess and tang |

### Ins

In [31]:
riddle_1 = """
I have five oranges, and the sum of what my sister and brother have is three times what I have plus thirty-five, answer in short form
""".strip()

riddle_2 = """
Total with what my family is 50 oranges. If we subtract one-fifth of the number from them, and add ten more oranges, how many oranges will there be in the end?, answer in short form
""".strip()

riddle_3 = """
When I was seven, my sister was twice my age. Now I am seventy years old, how old can my sister be?, answer in short form
""".strip()

print(llm([HumanMessage(content=riddle_1)]))
print(llm([HumanMessage(content=riddle_2)]))
print(llm([HumanMessage(content=riddle_3)]))



content='50' additional_kwargs={} example=False
content='50' additional_kwargs={} example=False
content='77' additional_kwargs={} example=False


## Way to Fix halosination using examples

In [ ]:
examples = [
    {
        "question": "When I was seven, my sister was twice my age. Now I am seventy years old, how old can my sister be?",
        "answer": "\n".join([
            "We will followup some questions to get the answer.",
            "Follow up: How old was your sister when you were seven?",
            "Intermediate answer: Twice, which mean 14 years.",
            "Follow up: What is the difference between your age and your sister's age?",
            "Intermediate answer: 14 years - 7 years = 7 years.",
            "Follow up: When you were seventy years old, how old would your sister be?",
            "Intermediate answer: my age (70) +  The difference between me and my sister's age (7) = 77 years.",
            "Final Answer: 77 years."
        ])
    },
    {
        "question": "I have five oranges, and the sum of what my sister and brother have is three times what I have plus thirty-five?",
        "answer": "\n".join([
            "We will followup some questions to get the answer.",
            "Follow up: How many oranges do you have?",
            "Intermediate answer: 5 oranges.",
            "Follow up: How many oranges does your sister and brother have?",
            "Intermediate answer: three times: = 3 * 5 = 15. Also, we need to add thirty-five to them: 15 + 35 = 50 ",
            "Final Answer: 50 oranges."
        ])
    }  
]

example_prompt = PromptTemplate(input_variables=["question", "answer"], template="Question: {question}\n{answer}")

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"]
)

message = prompt.format(input="Total with what my family is 50 oranges. If we subtract one-fifth of the number from them, and add ten more oranges, how many oranges will there be in the end?")
print( llm(message) )